In [167]:
import numpy as np
import pandas as pd
from stempel import StempelStemmer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import copy
from sklearn.feature_extraction.text import CountVectorizer

In [325]:
stemmer = StempelStemmer.polimorf()

Loading: 100%|██████████| 11368252/11368252 [00:16<00:00, 681387.32bytes/s] 


### Tworzenie listy stopwordsów

In [303]:
with open('data\stop_words_polish.txt', encoding="utf-8") as f:
    stopwords = f.readlines()

for i in range(len(stopwords)):
    stopwords[i] = stopwords[i].replace('\n', '')

In [324]:
grouped_emotions = pd.read_csv('data\grouped_emotions_dictionary.csv')
grouped_emotions.set_index('word', inplace=True)
grouped_emotions

,emotions
word,
$,0
$L,0
$T,0
"1,3-butadien",0
1/2,0
...,...
′,0
″,0
€,0


### Funkcje pomocnicze

In [326]:
def load_file(filepath):
    """
    Wczytuje plik tekstowy i zwraca jego zawartość jako string
    """
    with open(filepath, encoding="utf-8") as f:
        text = f.read()
    return text

def create_description_column(filepath, posting_delimiter='********'):
    """
    Potrzebne tylko kiedy wczytujemy dane z pliku tekstowego, w przypadku wczytywania z csv zamiast outputu tej funkcji podajemy df['description']
    """
    text = load_file(filepath)
    
    text_transformed = text.replace('\n', ' ').replace('\r', '')
    opisy_ofert = text_transformed.split(posting_delimiter)
    return opisy_ofert

def preprocess_text(description_column, stopwords, stemmer):
    """
    Usuwanie stopwordsów oraz stemming
    """
    
    opisy_ofert_bez_stopwords = []
    for opis in description_column:
        opis_bez_stopwords = opis
        for stopword in stopwords:
            opis_bez_stopwords = re.sub(r"\b%s\b" %stopword, '', opis_bez_stopwords)
        opisy_ofert_bez_stopwords.append(opis_bez_stopwords)

    opisy_po_stemmingu = []
    for opis in opisy_ofert_bez_stopwords:
        opisy_po_stemmingu.append([stemmer.stem(word.lower()) for word in opis.split()])

    return opisy_po_stemmingu


def create_tfidf_frame(opisy_ofert):
    """
    Tworzy ramkę danych ze słowami jako kolumnami i wartościami tfidf
    """

    vectorizer = TfidfVectorizer()
    cechy_tfidf = vectorizer.fit_transform(opisy_ofert)

    tfidf = cechy_tfidf.toarray()
    cechy_df = pd.DataFrame(tfidf, columns=vectorizer.get_feature_names_out())

    # Wyświetlanie ramki danych
    return cechy_df


def create_tfidf_columns(df_tfidf):
    tfidf_cols = pd.DataFrame([df_tfidf.iloc[:,:-1].sum(axis=1), df_tfidf.iloc[:,:-1].mean(axis=1)]).T
    tfidf_cols.columns=['tfidf_sum', 'tfidf_mean']
    return tfidf_cols

def create_emotions_columns(opisy_ofert, grouped_emotions):
    # Tworzenie wektora cech
    vectorizer = CountVectorizer()
    cechy = vectorizer.fit_transform(opisy_ofert)

    # Konwersja wektora cech do ramki danych
    df_emotions = pd.DataFrame(cechy.toarray(), columns=vectorizer.get_feature_names_out())

    for column in  list(df_emotions.columns):
        if column in list(grouped_emotions.index):
            df_emotions[column] *= grouped_emotions.loc[column,'emotions']

    emotion_cols = pd.DataFrame([df_emotions.sum(axis=1), df_emotions.mean(axis=1)]).T
    emotion_cols.columns=['emotions_sum', 'emotions_mean']
    return emotion_cols

def create_text_based_columns(df, text_column):
    df['text_length'] = df[text_column].apply(len)

    #liczba dużych liter
    df['capital_letters_count'] = df[text_column].apply(lambda x: sum(1 for c in x if c.isupper()))

    #zliczenia liczb
    df['numbers_count'] = df[text_column].apply(lambda x: sum(1 for c in x if c.isdigit()))

    #zliczenia znaków
    df['question_marks_count'] = df[text_column].apply(lambda x: x.count('!'))

    #zliczenia znaków walutowych
    df['currency_signs_count'] = df[text_column].apply(lambda x: x.count('$') + x.count('€') + x.count('£') + len(re.findall(r'\bzł\b|zł\b', x, re.IGNORECASE)))

        #liczba słów całych wielka litera
    df['capital_words_count'] = df[text_column].apply(lambda x: sum(1 for word in x.split() if word.isupper()))

    #możliwy adres email
    df['possible_email'] = df[text_column].apply(lambda x: ','.join([word for word in x.split() if '@' in word]))

    #możliwy adres rzeczywisty
    df['possible_address'] = df[text_column].apply(lambda x: ', '.join(re.findall(r'.{0,10}ul\..{0,10}', x)))

    #niepolskie słowa
    df['non_polish_char_count'] = df[text_column].apply(lambda x: sum(1 for char in ''.join(x) if char.strip() and (not char.isalpha() or char.lower() not in 'aąbcćdeęfghijklłmnńoópqrsśtuvwxyzźż')))

    #możliwy numer telefonu
    df['possible_phone_numbers'] = df[text_column].apply(lambda x: re.findall(r'\d+(?:[-\s]?\d+)+', x))
    df['possible_phone_numbers'] = df['possible_phone_numbers'].apply(lambda x: ', '.join(number for number in x if sum(1 for c in number if c.isdigit()) >= 9 and sum(1 for c in number if c.isdigit()) < 13))
    return df

def create_keywords_counter_column(df, keywords, output_column_name, ispreprocessed=False):
    # final_df = df_desc_preprocessed.merge(tfidf_cols, left_index=True, right_index=True)
    # final_df = final_df.merge(emotion_cols, left_index=True, right_index=True)
    # final_df = create_text_based_columns(final_df, 'description')


    # all_keywords = ["dowód" , "zawsze" , "nigdy" , "pesel" , "kryptowaluty" ,  "nic" , "wszystko" , "konto bankowe"]
    if ispreprocessed:
        text_column = 'preprocessed_description'
    else:  
        text_column = 'description'
    keywords_counter_col = [0]* len(df[text_column])

    for i in range(len(df['description'])):
        for keyword in keywords:
            keywords_counter_col[i] += len(re.findall(keyword, df[text_column][i].lower()))

    df[output_column_name] = keywords_counter_col
    return df

# Funkcja do tworzenia kolumn z pojedynczego tekstu

In [368]:
def process_single_row(single_posting_desc):
    df = pd.DataFrame([single_posting_desc], columns=["description"])
    other_cols = df.drop(columns=['description'])
    opisy = preprocess_text(df['description'].replace('\n', '').replace('\r', ''), stopwords, stemmer)
    opisy_full = []

    for opis in opisy:
        try:
            opisy_full.append(' '.join(opis))
        except:
            print(opis)
            opisy_full.append('INVALID_DATA')
            continue
    df_desc_preprocessed = pd.DataFrame([list(df['description']), opisy_full]).T
    df_desc_preprocessed.columns = ['description', 'preprocessed_description']
    df_desc_preprocessed['index'] = df_desc_preprocessed.index
    df_desc_preprocessed = df_desc_preprocessed[df_desc_preprocessed['preprocessed_description'] != 'INVALID_DATA'].reset_index(drop=True)

    df_tfidf = create_tfidf_frame(df_desc_preprocessed['preprocessed_description'])
    tfidf_cols = create_tfidf_columns(df_tfidf)
    emotion_cols = create_emotions_columns(df_desc_preprocessed['preprocessed_description'], grouped_emotions)


    final_df = df_desc_preprocessed.merge(tfidf_cols, left_index=True, right_index=True)
    final_df = final_df.merge(emotion_cols, left_index=True, right_index=True)
    final_df = create_text_based_columns(final_df, 'description')
    
    personal_info_keywords = ["dowód" , "pesel", "konto", "numer", "adres", "numer", "karty", "kredytowej"]
    inspiring_keywords = ["niepowtarzalna", "super", "dziś", "dzisiaj", "wyjątkowa", "ekstra", "zawsze", "nigdy", "nic", "wszystko"]
    money_related_keywords = ["gwarantowana", "zysk", "obrót", "zyski", "najlepsza", "gwarantowany", "najlepszy","kryptowaluty"]

    #all_keywords_stemmed = [stemmer.stem(word) for word in all_keywords]

    final_df = create_keywords_counter_column(final_df, personal_info_keywords, "personal_info_keywords_count")
    final_df = create_keywords_counter_column(final_df, inspiring_keywords, "inspiring_keywords_count")
    final_df = create_keywords_counter_column(final_df, money_related_keywords, "money_related_keywords_count")

    final_df = final_df.merge(other_cols, left_index=True, right_index=True)
    return final_df

# Funkcja do processingu całego dataframe'a z csv

In [350]:
def create_final_dataframe(filepath):

    df = pd.read_csv(filepath)
    other_cols = df.drop(columns=['description'])
    opisy = preprocess_text(df['description'].replace('\n', '').replace('\r', ''), stopwords, stemmer)
    opisy_full = []

    for opis in opisy:
        try:
            opisy_full.append(' '.join(opis))
        except:
            print(opis)
            opisy_full.append('INVALID_DATA')
            continue
    df_desc_preprocessed = pd.DataFrame([list(df['description']), opisy_full]).T
    df_desc_preprocessed.columns = ['description', 'preprocessed_description']
    df_desc_preprocessed['index'] = df_desc_preprocessed.index
    df_desc_preprocessed = df_desc_preprocessed[df_desc_preprocessed['preprocessed_description'] != 'INVALID_DATA'].reset_index(drop=True)

    df_tfidf = create_tfidf_frame(df_desc_preprocessed['preprocessed_description'])
    tfidf_cols = create_tfidf_columns(df_tfidf)
    emotion_cols = create_emotions_columns(df_desc_preprocessed['preprocessed_description'], grouped_emotions)

    final_df = df_desc_preprocessed.merge(tfidf_cols, left_index=True, right_index=True)
    final_df = final_df.merge(emotion_cols, left_index=True, right_index=True)
    final_df = create_text_based_columns(final_df, 'description')


    personal_info_keywords = ["dowód" , "pesel", "konto", "numer", "adres", "numer", "karty", "kredytowej"]
    inspiring_keywords = ["niepowtarzalna", "super", "dziś", "dzisiaj", "wyjątkowa", "ekstra", "zawsze", "nigdy", "nic", "wszystko"]
    money_related_keywords = ["gwarantowana", "zysk", "obrót", "zyski", "najlepsza", "gwarantowany", "najlepszy","kryptowaluty"]

    #all_keywords_stemmed = [stemmer.stem(word) for word in all_keywords]

    final_df = create_keywords_counter_column(final_df, personal_info_keywords, "personal_info_keywords_count")
    final_df = create_keywords_counter_column(final_df, inspiring_keywords, "inspiring_keywords_count")
    final_df = create_keywords_counter_column(final_df, money_related_keywords, "money_related_keywords_count")

    final_df = final_df.merge(other_cols, left_index=True, right_index=True)
    return final_df

In [355]:
final_df = create_final_dataframe('data/olx_1000.csv')
final_df = final_df.drop(columns=["user-profile-link-hash"])
final_df.to_csv("data/df_final_olx_v3.csv")


['dopldniowy', 'polska,', 'lider', 'rynek', 'usługi', 'kurierskich,', 'oddział', 'ełkustawa', 'poszukiwać', 'kurierów:', 'oferujemy:', 'współpraca', 'podstawa', 'umowy', 'kurierski', '(działalność', 'gospodarcza)gwarancja', 'stały', 'zleceńmożliwość', 'skorzystać', 'atrakcyjny', 'oferta', 'zakupić', 'ubranie', 'kurierskiegomożliwość', 'wynajm', 'samochód', 'dopldniowy', 'polski', 'preferencyjny', 'warunkachwesprzciee', 'założyć', 'działalność', 'gospodarczejdostęp', 'portal', 'kurierski', 'z', 'zniżka', 'partner', 'biznesowy', 'dpdmożliwość', 'objęcie', 'prywatny', 'opieka', 'medyczny', None, 'członek', 'rodzinymożliwość', 'skorzystać', 'program', 'multisportpakiet', 'szkoleńmożliwość', 'rozwój', 'zawodowy', 'daleki', 'stały', 'współpraca', 'struktura', 'dopldniowy', 'polski', 'oczekujemy:zaangażowania,', 'motywacja', 'działanie', 'dyspozycyjnościwysoki', 'kultura', 'osobisty', 'konieczny', 'kontakt', 'klient', 'dpdsumienność', 'rzetelnościsamodzielność', 'działanie', 'chęć', 'rozwojup

,description,preprocessed_description,index,tfidf_sum,tfidf_mean,emotions_sum,emotions_mean,text_length,capital_letters_count,numbers_count,...,user-profile-link,filters,pay_low,pay_high,pay_currency,pay_period,Lokalizacja,Wymiar pracy,Typ umowy,user-profile-link-hash
0,"Jesteś młodą, rzetelną osobą, chcącą podjąć si...","być młodą, rzetelny osobą, chcieć podjąć zarob...",0,13.767936,0.000794,204.0,0.011769,2952,256,17,...,https://www.olx.pl/oferty/uzytkownik/beuNf/,Doświadczenie nie jest wymagane/Elastyczny cza...,3000.0,7000.0,zł,mies. brutto,Wałbrzych,Dodatkowa / sezonowa,Inny,6643597317416472426
1,"Wyobraź sobie lata 70-te, niewielkie portowe m...","wyobrazić rok 70-, niewielki portowy miasto pó...",1,11.971166,0.000691,134.0,0.007730,1859,34,19,...,http://newyorker.olx.pl,Odpowiednie doświadczenie zawodowe/Praca zmian...,NaN,NaN,NaN,NaN,Wałbrzych,Pełny etat,Umowa o pracę,4076073661146739647
2,zatrudnimy kucharza z doświadczeniemOferujemy...,zatrudnić kucharz doświadczeniemoferujemy:- st...,2,6.101924,0.000352,33.0,0.001904,478,4,4,...,https://www.olx.pl/oferty/uzytkownik/3aaIp/,Odpowiednie doświadczenie zawodowe/Praca w wee...,25.0,45.0,zł,godz. brutto,Nieborowice,Pełny etat,Umowa o pracę,3919658943022206359
3,Firma z Koszalina zatrudni brukarza fachowca o...,firma koszalina zatrudnić brukarz fachowiec po...,3,5.949186,0.000343,33.0,0.001904,541,3,6,...,https://www.olx.pl/oferty/uzytkownik/h1Bf/,Odpowiednie doświadczenie zawodowe,4000.0,7000.0,zł,mies. brutto,Koszalin,Pełny etat,Inny,6441846835334410115
4,Firma Czysty Efekt jest polską firmą świadcząc...,firma czysty efekt polski firma świadczyć usłu...,4,12.068507,0.000696,131.0,0.007557,1994,57,31,...,http://czystyefekt.olx.pl,Odpowiednie doświadczenie zawodowe,NaN,NaN,NaN,NaN,"Kraków, Czyżyny",Pełny etat,Umowa o pracę,6558068349138217045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,PRACUJ Z NAMI! \r\n \r\nChcesz pracować w miej...,pracować ezyt nami! chcieć pracować miejsce uw...,989,8.886325,0.000513,97.0,0.005596,1276,44,9,...,http://zak.olx.pl,Odpowiednie doświadczenie zawodowe,NaN,NaN,NaN,NaN,"Wrocław, Stare Miasto",Pełny etat,Umowa o pracę,4623816692935245575
959,Biuro rachunkowe przyjmie na staż do księgowoś...,biuro rachunkowy przyjąć staż księgowość osoba...,990,4.436919,0.000256,18.0,0.001038,204,5,6,...,http://fairwind.olx.pl,Kraj: Pozostałe/Doświadczenie nie jest wymagane,NaN,NaN,NaN,NaN,"Szczecin, Centrum",Pełny etat,Inny,-5814310722019070226
960,Firma Futuro T&T Sp. z o.o. to pierwsza polska...,firma futuua t&tona sp. .. pierwszy polski pry...,991,10.501983,0.000606,120.0,0.006923,1783,40,0,...,https://www.olx.pl/oferty/uzytkownik/4hgvC/,Odpowiednie doświadczenie zawodowe/Elastyczny ...,4000.0,8000.0,zł,mies. brutto,"Warszawa, Praga-Północ",Pełny etat,Umowa o pracę,-4920627966913057383
961,"Karo-Megier Sp. z o.o. z siedzibą w Sątocznie,...","karo-megiera sp. .. siedziba sątocznie, 77-130...",992,5.212693,0.000301,22.0,0.001269,286,14,9,...,https://www.olx.pl/oferty/uzytkownik/4nbMG/,Odpowiednie doświadczenie zawodowe/Praca zmian...,3800.0,4500.0,zł,mies. brutto,Kołobrzeg,Pełny etat,Umowa o pracę,7404105014016741183


In [360]:
final_df

,description,preprocessed_description,index,tfidf_sum,tfidf_mean,emotions_sum,emotions_mean,text_length,capital_letters_count,numbers_count,...,category-tree-item,user-profile-link,filters,pay_low,pay_high,pay_currency,pay_period,Lokalizacja,Wymiar pracy,Typ umowy
0,"Jesteś młodą, rzetelną osobą, chcącą podjąć si...","być młodą, rzetelny osobą, chcieć podjąć zarob...",0,13.767936,0.000794,204.0,0.011769,2952,256,17,...,Strona główna/Praca/Marketing i PR/Media i PR/...,https://www.olx.pl/oferty/uzytkownik/beuNf/,Doświadczenie nie jest wymagane/Elastyczny cza...,3000.0,7000.0,zł,mies. brutto,Wałbrzych,Dodatkowa / sezonowa,Inny
1,"Wyobraź sobie lata 70-te, niewielkie portowe m...","wyobrazić rok 70-, niewielki portowy miasto pó...",1,11.971166,0.000691,134.0,0.007730,1859,34,19,...,"Strona główna/Praca/Pracownik sklepu/Kasjer, p...",http://newyorker.olx.pl,Odpowiednie doświadczenie zawodowe/Praca zmian...,NaN,NaN,NaN,NaN,Wałbrzych,Pełny etat,Umowa o pracę
2,zatrudnimy kucharza z doświadczeniemOferujemy...,zatrudnić kucharz doświadczeniemoferujemy:- st...,2,6.101924,0.000352,33.0,0.001904,478,4,4,...,Strona główna/Praca/Gastronomia/Kucharz/Kuchar...,https://www.olx.pl/oferty/uzytkownik/3aaIp/,Odpowiednie doświadczenie zawodowe/Praca w wee...,25.0,45.0,zł,godz. brutto,Nieborowice,Pełny etat,Umowa o pracę
3,Firma z Koszalina zatrudni brukarza fachowca o...,firma koszalina zatrudnić brukarz fachowiec po...,3,5.949186,0.000343,33.0,0.001904,541,3,6,...,Strona główna/Praca/Budowa / remonty/Budowa / ...,https://www.olx.pl/oferty/uzytkownik/h1Bf/,Odpowiednie doświadczenie zawodowe,4000.0,7000.0,zł,mies. brutto,Koszalin,Pełny etat,Inny
4,Firma Czysty Efekt jest polską firmą świadcząc...,firma czysty efekt polski firma świadczyć usłu...,4,12.068507,0.000696,131.0,0.007557,1994,57,31,...,Strona główna/Praca/Sprzątanie/Sprzątanie - Ma...,http://czystyefekt.olx.pl,Odpowiednie doświadczenie zawodowe,NaN,NaN,NaN,NaN,"Kraków, Czyżyny",Pełny etat,Umowa o pracę
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,PRACUJ Z NAMI! \r\n \r\nChcesz pracować w miej...,pracować ezyt nami! chcieć pracować miejsce uw...,989,8.886325,0.000513,97.0,0.005596,1276,44,9,...,Strona główna/Praca/Edukacja/Szkolnictwo/Szkol...,http://zak.olx.pl,Odpowiednie doświadczenie zawodowe,NaN,NaN,NaN,NaN,"Wrocław, Stare Miasto",Pełny etat,Umowa o pracę
959,Biuro rachunkowe przyjmie na staż do księgowoś...,biuro rachunkowy przyjąć staż księgowość osoba...,990,4.436919,0.000256,18.0,0.001038,204,5,6,...,Strona główna/Praca/Praca za granicą/Praca za ...,http://fairwind.olx.pl,Kraj: Pozostałe/Doświadczenie nie jest wymagane,NaN,NaN,NaN,NaN,"Szczecin, Centrum",Pełny etat,Inny
960,Firma Futuro T&T Sp. z o.o. to pierwsza polska...,firma futuua t&tona sp. .. pierwszy polski pry...,991,10.501983,0.000606,120.0,0.006923,1783,40,0,...,Strona główna/Praca/IT / telekomunikacja/Zarzą...,https://www.olx.pl/oferty/uzytkownik/4hgvC/,Odpowiednie doświadczenie zawodowe/Elastyczny ...,4000.0,8000.0,zł,mies. brutto,"Warszawa, Praga-Północ",Pełny etat,Umowa o pracę
961,"Karo-Megier Sp. z o.o. z siedzibą w Sątocznie,...","karo-megiera sp. .. siedziba sątocznie, 77-130...",992,5.212693,0.000301,22.0,0.001269,286,14,9,...,Strona główna/Praca/Hotelarstwo/Obsługa hotelo...,https://www.olx.pl/oferty/uzytkownik/4nbMG/,Odpowiednie doświadczenie zawodowe/Praca zmian...,3800.0,4500.0,zł,mies. brutto,Kołobrzeg,Pełny etat,Umowa o pracę


In [358]:
final_df_sprzedajemy = create_final_dataframe('data/sprzedajemy_1000.csv')

['witać', 'przyjąć', 'prace', 'produkcja', 'najlepiej', 'systema', '8', 'godzina', ',', 'możliwość', 'ustalić', 'grafik', None, ',', 'nocka', ',szukać', 'praca', 'dorywczy', 'produkcja', 'taśma', 'magazyn', ',', 'posiadać', 'uprawnienie', 'wózek', 'widłowy', '(udt)', 'podest', 'ruchomy', ',', 'pracowity', ',', 'komunikatywny', ',', 'koleżeński', ',punktualny', 'szybki', 'uczyć', '.n', 'interesować', 'oferta', 'przedstawiciel', 'jakiś', 'marka', ',', 'promocja', 'produkt', '.zapraszać', 'kontakt', ':)']
['szukać', 'absolwent', 'studium', 'wyżesoki', 'podyplomowych,', 'ukończyć', 'studium', '12.2017', 'r.', 'student', 'ostatnie', 'studium', '(licencjackich,', 'magisterskich,', 'doktorancki', 'podyplomowych)', 'stanowisko:', 'tester', 'oprogramowanie', 'wymagania:', 'status', 'student', 'ostatnie', 'studium', 'absolwenta,', 'orzeczenie', 'stopień', 'niepełnosprawności,', 'pozostawać', 'zatrudnienie', 'oferujemy:', '-', 'szkolić', 'zakres', 'testować', 'oprogramowanie', '-', 'kilkumiesięcz

In [366]:
final_final_df = pd.concat([final_df,final_df_sprzedajemy])
final_final_df.to_csv("data/final_final_df.csv")

In [336]:
opisy_pozytywne_desc = create_description_column('data\FakeJobHunter - pozytywne.txt',"Opis ogłoszenia")
opisy_negatywne_desc = create_description_column('data\FakeJobHunter - negatywne.txt')

opisy_pozytywne = preprocess_text(opisy_pozytywne_desc, stopwords, stemmer)
opisy_negatywne = preprocess_text(opisy_negatywne_desc, stopwords, stemmer)

In [337]:
opisy_pozytywne_full = []
opisy_negatywne_full = []

for opis in opisy_pozytywne:
    try:
        opisy_pozytywne_full.append(' '.join(opis))
    except:
        print(opis)
        opisy_pozytywne_full.append('INVALID_DATA')
        continue

for opis in opisy_negatywne:
    try:
        opisy_negatywne_full.append(' '.join(opis))
    except:
        print(opis)
        opisy_negatywne_full.append('INVALID_DATA')
        continue

['nativus', 'dzień', 'doradca:', 'mieć', 'bezpośredni', 'kontakt', 'nasze', 'klient', 'zapewniać', 'kompleksowy', 'obsługę,', 'być', 'profesjonalny', 'budować', 'przyjazny', 'relacja', 'klientami,', 'pracować', 'swój', 'premię,', 'prowadzić', 'aktywny', 'sprzedaż', 'produkt', 'usługi', 'bankowy', 'ubezpieczeniowych,', 'przeprowadzać', 'transakcja', 'gotówkowy', 'bezgotówkowe.', 'to', 'stanowisko', 'twoje,', ':', 'interesować', 'branża', 'finansowy', '–', 'chcieć', 'nenen', 'wiedza', 'rozwijać,', 'lubić', 'ludzi.', 'n', 'bać', 'kontakt', 'klient', 'być', 'otwarty/', 'potrzeby,', 'podchodzić', 'entuzjastycznić', 'praca', 'chcieć', 'mść', 'satysfakcja', 'osiągać', 'celów,', 'mieć', 'wykształcić', 'średnia', 'wyższe,', 'mieć', 'doświadczyć', 'praca', 'związać', 'finansami,', 'sprzedaż', 'obsługa', 'klient', '(mila', 'widziane).', 'dołączać', 'otrzymasz:', 'umowa', 'pracę,', 'stały', 'wynagrodzić', 'premię,', 'atrakcyjny', 'pakiet', 'benefit', 'karta', 'multisport,', 'prywatny', 'opieka', '

In [338]:
len(opisy_pozytywne_full), len(opisy_negatywne_full)

(14, 25)

In [339]:
etykiety = [1] * len(opisy_pozytywne_full) + [0] * len(opisy_negatywne_full)
opisy_ofert_full = opisy_pozytywne_full + opisy_negatywne_full
opisy_ofert_desc = opisy_pozytywne_desc + opisy_negatywne_desc



df_desc_preprocessed = pd.DataFrame([opisy_ofert_desc, opisy_ofert_full]).T
df_desc_preprocessed.columns = ['description', 'preprocessed_description']
df_desc_preprocessed['label'] = etykiety
df_desc_preprocessed['index'] = df_desc_preprocessed.index
df_desc_preprocessed = df_desc_preprocessed[df_desc_preprocessed['preprocessed_description'] != 'INVALID_DATA'].reset_index(drop=True)
df_desc_preprocessed


,description,preprocessed_description,label,index
0,,,1,0
1,SAMODZIELNA KSIĘGOWA Twój zakres obowiązków ...,samodzielny księgowy twój zakres obowiązek ksi...,1,1
2,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,2
3,"Zatrudnimy doracę, doradczynię ds. kontaktu z...","zatrudnić doracę, doradczyni ds. kontakt klien...",1,3
4,Jeśli: w pracy nie lubisz rutyny szukasz sta...,jeśli: praca lubić rutyna szukać stały dodatko...,1,4
5,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,5
6,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,6
7,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,7
8,"Szukam ambitnych osób do zespołu, które nie b...","szukać ambitny osoba zespołu, boja nowy wyzwań...",1,8
9,LUBISZ DORADZAĆ INNYM? RÓB TO Z NAMI! Doradza...,lubić doradzać innym? robić to ezyt nami! dora...,1,9


In [340]:

df_tfidf = create_tfidf_frame(df_desc_preprocessed['preprocessed_description'])
tfidf_cols = create_tfidf_columns(df_tfidf)
tfidf_cols


,tfidf_sum,tfidf_mean
0,0.000000,0.000000
1,10.145443,0.006282
2,11.200993,0.006936
3,3.341539,0.002069
4,5.797564,0.003590
5,9.428065,0.005838
6,9.375801,0.005805
7,9.420874,0.005833
8,6.901856,0.004274
9,9.819691,0.006080


In [341]:
grouped_emotions = pd.read_csv('data\grouped_emotions_dictionary.csv')
grouped_emotions.set_index('word', inplace=True)
grouped_emotions

,emotions
word,
$,0
$L,0
$T,0
"1,3-butadien",0
1/2,0
...,...
′,0
″,0
€,0


In [342]:
# UWAGA - długo się wykonuje
emotion_cols = create_emotions_columns(df_desc_preprocessed['preprocessed_description'], grouped_emotions)
emotion_cols

,emotions_sum,emotions_mean
0,0.0,0.000000
1,100.0,0.061881
2,117.0,0.072401
3,10.0,0.006188
4,34.0,0.021040
5,82.0,0.050743
6,71.0,0.043936
7,77.0,0.047649
8,48.0,0.029703
9,86.0,0.053218


In [343]:
final_df = df_desc_preprocessed.merge(tfidf_cols, left_index=True, right_index=True)
final_df = final_df.merge(emotion_cols, left_index=True, right_index=True)
final_df = create_text_based_columns(final_df, 'description')


personal_info_keywords = ["dowód" , "pesel", "konto", "numer", "adres", "numer", "karty", "kredytowej"]
inspiring_keywords = ["niepowtarzalna", "super", "dziś", "dzisiaj", "wyjątkowa", "ekstra", "zawsze", "nigdy", "nic", "wszystko"]
money_related_keywords = ["gwarantowana", "zysk", "obrót", "zyski", "najlepsza", "gwarantowany", "najlepszy","kryptowaluty"]

#all_keywords_stemmed = [stemmer.stem(word) for word in all_keywords]

final_df = create_keywords_counter_column(final_df, personal_info_keywords, "personal_info_keywords_count")
final_df = create_keywords_counter_column(final_df, inspiring_keywords, "inspiring_keywords_count")
final_df = create_keywords_counter_column(final_df, money_related_keywords, "money_related_keywords_count")
final_df


,description,preprocessed_description,label,index,tfidf_sum,tfidf_mean,emotions_sum,emotions_mean,text_length,capital_letters_count,...,question_marks_count,currency_signs_count,capital_words_count,possible_email,possible_address,non_polish_char_count,possible_phone_numbers,personal_info_keywords_count,inspiring_keywords_count,money_related_keywords_count
0,,,1,0,0.000000,0.000000,0.0,0.000000,0,0,...,0,0,0,,,0,,0,0,0
1,SAMODZIELNA KSIĘGOWA Twój zakres obowiązków ...,samodzielny księgowy twój zakres obowiązek ksi...,1,1,10.145443,0.006282,100.0,0.061881,1657,51,...,0,0,5,,,31,,0,1,1
2,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,2,11.200993,0.006936,117.0,0.072401,1744,40,...,1,0,3,,,49,,1,1,1
3,"Zatrudnimy doracę, doradczynię ds. kontaktu z...","zatrudnić doracę, doradczyni ds. kontakt klien...",1,3,3.341539,0.002069,10.0,0.006188,128,4,...,0,0,0,,,5,,0,0,0
4,Jeśli: w pracy nie lubisz rutyny szukasz sta...,jeśli: praca lubić rutyna szukać stały dodatko...,1,4,5.797564,0.003590,34.0,0.021040,416,3,...,0,0,0,,,2,,0,0,0
5,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,5,9.428065,0.005838,82.0,0.050743,1316,31,...,1,0,0,,,34,,0,1,1
6,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,6,9.375801,0.005805,71.0,0.043936,1075,28,...,1,1,2,,,40,,0,1,1
7,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,7,9.420874,0.005833,77.0,0.047649,1082,26,...,1,1,1,,,41,,0,1,1
8,"Szukam ambitnych osób do zespołu, które nie b...","szukać ambitny osoba zespołu, boja nowy wyzwań...",1,8,6.901856,0.004274,48.0,0.029703,579,13,...,0,0,0,,,14,,0,0,0
9,LUBISZ DORADZAĆ INNYM? RÓB TO Z NAMI! Doradza...,lubić doradzać innym? robić to ezyt nami! dora...,1,9,9.819691,0.006080,86.0,0.053218,1217,77,...,2,0,14,,,42,,0,0,2


In [344]:
final_df.to_csv("data\example_data_v2.csv")

In [370]:
process_single_row("afeafaef eafadADAfeafe c aeafage")

,description,preprocessed_description,index,tfidf_sum,tfidf_mean,emotions_sum,emotions_mean,text_length,capital_letters_count,numbers_count,question_marks_count,currency_signs_count,capital_words_count,possible_email,possible_address,non_polish_char_count,possible_phone_numbers,personal_info_keywords_count,inspiring_keywords_count,money_related_keywords_count
0,afeafaef eafadADAfeafe c aeafage,afeafaef eafadadafeafe córka aeafage,0,1.5,0.5,4.0,1.0,32,3,0,0,0,0,,,0,,0,0,0
